# Sujet 3 : Programmation dynamique (TP)

La programmation dynamique consiste à diviser un problème en sous-problèmes dont les solutions permettent de résoudre l'instance principale. Il est donc nécessaire de trouver une relation de récurrence entre la solution d'un problèmme et celles de ses sous-problèmes.


In [8]:
price=Int64[42,40,12,25]
weight=Int64[7,4,3,5]
cap=10

10

In [31]:
function KnapsackDynamic(price, weight, cap)
    n = length(weight)
    C = [0 for i=1:cap+1, j=1:n+1]
    for i = 2:n+1
        #println("col: ",i)
        for j = 1:cap+1
            #println("ligne: ",j)
            if j > weight[i-1]
                C[j,i] = max(C[j-weight[i-1], i-1] + price[i-1], C[j, i-1])

            else
                C[j,i] = C[j,i-1]
            end

        end
    end
    return C
end

function ObjectsTaken(C, price, weight)
    j,i = size(C)
    O = []
    while i > 1 && j > 0
        # Vérifie si l'objet a été pris, se base sur notre idée initiale de choisir pendant la création de C
        if j  > weight[i - 1] && C[j, i] == C[j - weight[i - 1], i - 1] + price[i - 1]
            push!(O, i - 1)
            j = j - weight[i - 1]
        end
        i = i - 1
    end
    n1, n2 = size(C)
    return O, C[n1, n2]
end

ObjectsTaken (generic function with 1 method)

L'algorithme utilise une matrice $C$ bidimensionnelle qu'il va remplir colonne par colonne. L'élément $C_{j,i}$ représente la valeur maximale atteignable en se limitant aux $i$ premiers objets et à une capacité maximale de $j$. 

$C$ est initialisée avec des $0$.

Pour chaque colonne (objets), on parcourt les lignes (capacités max considérées), deux possibilités :
- La capacité actuelle est inférieure au poids de l'objet considéré
    
    Dans ce cas, on doit choisir d'ajouter ou non l'objet à notre solution.
    On a
    - $C_{j-w_i, i} + v_i$ : valeur maximale atteignable si on ajoute l'objet $i$
    - $C_{j,i-1}$ : valeur maximale atteignable sans l'objet $i$
    On met à jour $C_{j,i}$ avec le maximum de ces deux valeurs

- L'objet a un poids trop élevé pour être ajouté, dans ce cas, on a $C_{j,i}=C_{j,i-1}$.

    On utilise la meilleure solution du sous-problème à $i-1$ objets.

On renvoie finalement la matrice $C$. La solution optimale du problème se trouve en $C_{cap, n}$ qui représente la solution du problème initial avec une capacité de $cap$ et $n$ objets.


In [32]:
C = KnapsackDynamic(price, weight, cap)
ObjectsTaken(C, price, weight)

(Any[4, 2], 65)

On rappelle l'instance utilisée
|       | 1   | 2   | 3   | 4   |
| ----- | --- | --- | --- | --- |
| $c_i$ | 42  | 40  | 12  | 25  |
| $w_i$ | 7   | 4   | 3   | 5   |

et $C=10$

La solution calculée par l'algorithme est de choisir les objets $4$ et $2$ pour une valeur totale de $40+25=65$ et un poids de $4+5=9<=C$.

On peut vérifier rapidement l'optimalité de cette solution.
Si on choisit de prendre l'objet $1$, la capacité restante est $C-w_1=10-7=3$, on peut ensuite seulement choisir l'objet $3$ pour une valeur totale de $42+12=54 < 65$ non optimale.

Symétriquement, si on choisir l'objet $3$, choisir ensuite l'objet $7$ ne produit pas une solution optimale, de même que choisir les objets $2$ ($12+40=52<65$) ou $4$ ($12+25=37<65$)

Il ne reste que la solution calculée $2$ et $4$ qui est valide.

La solution renvoyée est bien optimale.

In [59]:
function readKnaptxtInstance(filename)
    price=Int64[]
    weight=Int64[]
    KnapCap=Int64[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

function solveKnapInstance(filename)
        println(filename)
    price, weight, cap = readKnaptxtInstance(filename)
    C = KnapsackDynamic(price, weight, cap)
    objects, bestSol = ObjectsTaken(C, price, weight)

    w1, _ = size(C)
    println("Pour un poids maximum de : ", w1-1)
    println("La meilleure solution est ", bestSol, ", pour les objets : ")
    for i in objects
        println("\$ ",i, " -> de poids ", weight[i], " et de valeur : ", price[i])
    end
    printstyled("-------------------------------------------------------------------------\n", bold=true, color=:red)
end

solveKnapInstance (generic function with 1 method)

In [61]:
solveKnapInstance("../TP2/InstancesKnapSack/test.opb.txt")
solveKnapInstance("../TP2/InstancesKnapSack/Similar_Weights/KnapSack_1000_1000_-8943.opb.txt")
solveKnapInstance("../TP2/InstancesKnapSack/Similar_Weights/KnapSack_100_1000_-995.opb.txt")
solveKnapInstance("../TP2/InstancesKnapSack/Strongly_Correlated/KnapSack_100_1000_-2397.opb.txt")

../TP2/InstancesKnapSack/test.opb.txt
Pour un poids maximum de : 10
La meilleure solution est 65, pour les objets : 
$ 4 -> de poids 5 et de valeur : 25
$ 2 -> de poids 4 et de valeur : 40
-------------------------------------------------------------------------
../TP2/InstancesKnapSack/Similar_Weights/KnapSack_1000_1000_-8943.opb.txt
Pour un poids maximum de : 990583
La meilleure solution est 8943, pour les objets : 
$ 889 -> de poids 100080 et de valeur : 991
$ 858 -> de poids 100065 et de valeur : 1000
$ 814 -> de poids 100042 et de valeur : 996
$ 811 -> de poids 100003 et de valeur : 990
$ 782 -> de poids 100046 et de valeur : 991
$ 559 -> de poids 100042 et de valeur : 991
$ 273 -> de poids 100007 et de valeur : 997
$ 77 -> de poids 100085 et de valeur : 992
$ 29 -> de poids 100093 et de valeur : 995
-------------------------------------------------------------------------
../TP2/InstancesKnapSack/Similar_Weights/KnapSack_100_1000_-995.opb.txt
Pour un poids maximum de : 100099
La 